In [1]:
import os
import zipfile
import random
import shutil
from shutil import copyfile
import cv2.cv2 as cv2

import tensorflow as tf
import numpy as np
import matplotlib as plt
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from stn import spatial_transformer_network as transformer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas
from tensorflow.keras.applications import InceptionV3

%precision 4

'%.4f'

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(20, (2,2), activation="relu", input_shape=(256,256,3), strides=2),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(50, (2,2), activation="relu", strides=2),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(500, activation="relu"),
    tf.keras.layers.Dense(7, activation="softmax"),
])

In [ ]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['acc'])


In [ ]:
def convert_to_grayscale(img):
      return tf.image.rgb_to_grayscale(img, name=None)


TRAINING_DIR = "emotions/train/"
train_datagen = ImageDataGenerator(
      rescale=1./255,
      preprocessing_function=convert_to_grayscale,)
      # rotation_range=40,
      # width_shift_range=0.2,
      # height_shift_range=0.2,
      # shear_range=0.2,
      # zoom_range=0.2,
      # horizontal_flip=True,
      # fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, target_size=(256,256), 
batch_size=32, class_mode="categorical")

VALIDATION_DIR = "emotions/validation/"
validation_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=convert_to_grayscale,)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, target_size=(256,256),batch_size=32, class_mode="categorical")

In [2]:
model = tf.keras.models.load_model("xyz.model")

In [3]:
test_dir = "emotions/test/"
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(256,256), 
batch_size=32, class_mode="categorical")

model.evaluate(test_generator)

Found 3343 images belonging to 7 classes.
105/105 [==============================] - 7s 71ms/step - loss: 2.8084e-04 - acc: 1.0000


[0.0003, 1.0000]

In [4]:
class Faces:
    def __init__(self):
        self.model = model

    
    def find_face(self):
        face_cascade = cv2.CascadeClassifier("hc.xml")
        path = "face_test/"
        cap = cv2.VideoCapture(0)
        count = 1
        while True:
            face_cascade = cv2.CascadeClassifier("hc.xml")
            path = "face_test/"
            cap = cv2.VideoCapture(0)
            count = 1
            while True:
                ret, img = cap.read()
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                
                for (x,y,w,h) in faces:
                    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
                    face = gray[y:y+h, x:x+w]
                    print(self.predict(face))
                
                    # txt = "face_"+str(count)+".jpg"
                    # cv2.imwrite(path+txt, face)
                    # count += 1
                    # if count > 300:
                    #     count = 1

                cv2.imshow("img", img)
                k = cv2.waitKey(30)
                if k == 27:
                    break
        cap.release()
        cv2.destroyAllWindows()
    
    
    def predict(self, face):
        size = 256
        img_array = cv2.resize(face, (size,size))
        image = np.expand_dims(img_array, axis=0)
        image = np.expand_dims(img_array, axis=-1)
        zeros = np.ones((1,1,1,3))
        fi = image*zeros
        # image = img_array.reshape(-1, size,size,1)
        metrics = self.model.predict([fi])
        return metrics

In [5]:
xyz = Faces()
xyz.find_face()

[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0.]]


KeyboardInterrupt: 